In [112]:
import torch
import torch_pruning as tp
from yolox.models import yolox_tiny

model = yolox_tiny(num_classes=20,).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [113]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [114]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [115]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [116]:

temp = ['backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv',
'backbone.backbone.dark4[1].m[0].conv1.conv','backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv2.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv2.conv'
,'backbone.backbone.dark5[1].conv1.conv', 'backbone.backbone.dark5[1].conv2.conv',
'backbone.backbone.dark5[2].conv1.conv', 'backbone.backbone.dark5[2].conv2.conv', 
'backbone.backbone.dark5[2].m[0].conv1.conv','backbone.backbone.dark5[2].m[0].conv2.conv' ]



temp = ['backbone.backbone.dark3[1].conv1.conv','backbone.backbone.dark3[1].conv2.conv',
'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 
'backbone.backbone.dark3[1].m[2].conv1.conv',
'backbone.backbone.dark3[1].conv3.conv', ]
for i in range(len(temp)):
    layer = temp[i]
    pruning_idxs = strategy(eval(f"model.{layer}.weight"), amount=0.11) # or pruning_idxs=[2, 6, 9, ...]
    print(pruning_idxs)
    layer_split = layer
    pruning_plan = DG.get_pruning_plan(eval(f"model.{layer}"), tp.prune_conv_out_channel, idxs=pruning_idxs )
    print(pruning_plan)
    print(layer)

    # 4. execute this plan after checking (prune the model)
    #    if the plan prunes some channels to zero, 
    #    DG.check_pruning plan will return False.
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()
        
temp = ['backbone.backbone.dark4[1].conv1.conv','backbone.backbone.dark4[1].conv2.conv',
'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv',
'backbone.backbone.dark4[1].conv3.conv', ]
for i in range(len(temp)):
    layer = temp[i]
    pruning_idxs = strategy(eval(f"model.{layer}.weight"), amount=0.105) # or pruning_idxs=[2, 6, 9, ...]
    print(pruning_idxs)
    layer_split = layer
    pruning_plan = DG.get_pruning_plan(eval(f"model.{layer}"), tp.prune_conv_out_channel, idxs=pruning_idxs )
    print(pruning_plan)
    print(layer)

    # 4. execute this plan after checking (prune the model)
    #    if the plan prunes some channels to zero, 
    #    DG.check_pruning plan will return False.
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

temp = ['backbone.backbone.dark5[2].conv1.conv','backbone.backbone.dark5[2].conv2.conv',
'backbone.backbone.dark5[2].m[0].conv1.conv','backbone.backbone.dark5[2].m[0].conv2.conv',
'backbone.backbone.dark5[2].conv3.conv', ]
for i in range(len(temp)):
    layer = temp[i]
    pruning_idxs = strategy(eval(f"model.{layer}.weight"), amount=0.105) # or pruning_idxs=[2, 6, 9, ...]
    print(pruning_idxs)
    layer_split = layer
    pruning_plan = DG.get_pruning_plan(eval(f"model.{layer}"), tp.prune_conv_out_channel, idxs=pruning_idxs )
    print(pruning_plan)
    print(layer)

    # 4. execute this plan after checking (prune the model)
    #    if the plan prunes some channels to zero, 
    #    DG.check_pruning plan will return False.
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

temp = ['backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv','backbone.C3_p4.conv2.conv',
'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m[0].conv1.conv', 'backbone.C3_p4.m[0].conv2.conv',
'backbone.reduce_conv1.conv',
'backbone.bu_conv2.conv', 
'backbone.bu_conv1.conv', ]
for i in range(len(temp)):
    layer = temp[i]
    pruning_idxs = strategy(eval(f"model.{layer}.weight"), amount=0.105) # or pruning_idxs=[2, 6, 9, ...]
    print(pruning_idxs)
    layer_split = layer
    pruning_plan = DG.get_pruning_plan(eval(f"model.{layer}"), tp.prune_conv_out_channel, idxs=pruning_idxs )
    print(pruning_plan)
    print(layer)

    # 4. execute this plan after checking (prune the model)
    #    if the plan prunes some channels to zero, 
    #    DG.check_pruning plan will return False.
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()         


torch.save(model, 'model.pth') 



[12, 18, 23, 43, 44]
Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[12, 18, 23, 43, 44], metric={'#params': 480}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[12, 18, 23, 43, 44], metric={'#params': 10}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)) => Elemen

In [117]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz